In [1]:
import pandas as pd

from os.path import join

DATA_DIR = '/home/qke100/su-data/academic-tree/'

## Connect DF

In [2]:
! head -6 /home/qke100/su-data/academic-tree/connect.csv

"cid","pid1","pid2","relation","location","locid","addedby","dateadded"
"2","2","3","1","University of California, Berkeley","312","david","2005-01-15 17:33:57"
"3","4","3","2","University of California, Berkeley","312","david","2005-01-15 17:35:17"
"5","6","3","1","University of California, Berkeley","312","david","2005-01-15 19:02:54"
"6","18761","9","1","University of California, Berkeley","312","david","2005-01-15 19:06:16"
"7","10","16","2","Washington University, Saint Louis","497","david","2005-01-15 19:06:58"


In [3]:
conn_df = pd.read_csv(join(DATA_DIR, 'connect.csv'), sep=',').drop(columns=['addedby', 'dateadded'])
conn_df['pid1'] = conn_df['pid1'].astype(str)
conn_df['pid2'] = conn_df['pid2'].astype(str)
conn_df.shape

(1535285, 6)

In [4]:
conn_df.dtypes

cid          int64
pid1        object
pid2        object
relation     int64
location    object
locid        int64
dtype: object

In [5]:
ds_people = pd.read_hdf('dataset/people_df.h5')
ds_people.shape

(759676, 10)

In [6]:
pids = set(ds_people.pid)
len(pids)

759676

In [31]:
conn_subset = conn_df[(conn_df.pid1.isin(pids)) & (conn_df.pid2.isin(pids))]
conn_subset.shape

(723225, 6)

In [32]:
conn_subset[conn_subset.cid == 110948]

,cid,pid1,pid2,relation,location,locid
211296,110948,86769,5607,1,"Ohio State University, Columbus",223
301362,110948,86769,5607,1,"Ohio State University, Columbus",223


In [33]:
conn_subset[conn_subset.cid == 106030]

,cid,pid1,pid2,relation,location,locid
86562,106030,82907,82908,1,"IIT Kanpur, IISER Kolkata",18466
1520475,106030,82907,82908,1,IACS Kolkata,18467


In [34]:
conn_subset[conn_subset.cid == 426298]

,cid,pid1,pid2,relation,location,locid
360872,426298,198224,195226,1,"University of Nevada, Reno",410
360873,426298,198223,195767,1,Georgia Institute of Technology,995


In [35]:
conn_subset = conn_subset.drop_duplicates(subset=['cid', 'pid1', 'pid2'])
conn_subset.shape

(722896, 6)

In [36]:
dup_cids = set(k for k, v in conn_subset.cid.value_counts().items() if v > 1)
len(dup_cids)

12

In [37]:
ds_conn = conn_subset[~conn_subset.cid.isin(dup_cids)].copy().reset_index(drop=True)
ds_conn.shape

(722872, 6)

In [38]:
ds_conn.cid.nunique()

722872

In [41]:
ds_conn.to_hdf('dataset/connect_df.h5', key='df', mode='w')

/home/qke100/.local/lib/python3.6/site-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['pid1', 'pid2', 'location'], dtype='object')]

  encoding=encoding,


In [42]:
ds_conn.to_csv('dataset/connect.csv', index=False)

In [43]:
pids_conn = set(ds_conn.pid1) | set(ds_conn.pid2)
len(pids_conn)

724494